Kaggle Link: https://www.kaggle.com/lorenzozanolin/object-recognition-zanolin
        
W&B: https://wandb.ai/lorenzozanolin-52/object_recognition/table?workspace=user-lorenzozanolin-52

In [16]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from tqdm.notebook import trange, tqdm
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("a")
wandb.login(key=secret_value_0)
wandb.init(project='object_recognition', save_code=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


wandb: WARNING No program path found when generating artifact job source for a non-colab notebook run. See https://docs.wandb.ai/guides/launch/create-job
wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


batch_size,▁
epochs,▁
lr,▁
train_loss,█▅▃▃▂▂▂▁▁▁
val_accuracy,▁▄▅▆▅▆█▇▇█
val_loss,█▅▄▃▄▃▁▂▁▁
batch_size,64
epochs,10
lr,0.001
optimizer,Adam
train_loss,0.91772


## Import everything needed

In [17]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import collections
import math
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Unzip datasets

In [18]:
!pip install py7zr

# WARNING: It can take a lot of time to uncompress!

In [19]:
!python -m py7zr x /kaggle/input/cifar-10/train.7z

In [20]:
!python -m py7zr x /kaggle/input/cifar-10/test.7z

In [21]:
data_dir = '/kaggle/working/'

In [22]:
def read_csv_labels(fname):
    """Read `fname` to return a filename to label dictionary."""
    with open(fname, 'r') as f:
        # Skip the file header line (column name)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, '/kaggle/input/cifar-10/trainLabels.csv'))
print(f'Number training examples: {len(labels)}')
print(f'Number classes: {len(set(labels.values()))}')

Number training examples: 50000
Number classes: 10


In [23]:
def copyfile(filename, target_dir):
    """Copy a file into a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """Split the validation set out of the original training set."""
    # The number of examples of the class that has the fewest examples in the
    # training dataset
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # The number of examples per class for the validation set
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

def reorg_test(data_dir):
    """Organize the testing set for data loading during prediction."""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))
        
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels('/kaggle/input/cifar-10/trainLabels.csv')
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [24]:
batch_size = 64
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)
wandb.log({'batch_size': batch_size})

### Transformations
We will do some Image Augmentation:

In [25]:
transform_train = torchvision.transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #Random crop can help to make the NN invariant to the position of the element
    transforms.RandomHorizontalFlip(),  #we will use horizontal flip
    torchvision.transforms.ToTensor(), #transform the image into a tensor
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],  #each channel (3) will be normalized using the mean (first tuple) and the std (second tuple)
                                     [0.2023, 0.1994, 0.2010])
])

transform_test = torchvision.transforms.Compose([  #since it is the dataset, we will not do data augmentation, other than normalization
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], #each channel (3) will be normalized using the mean (first tuple) and the std (second tuple)
                                     [0.2023, 0.1994, 0.2010])]) 

In [26]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_test) for folder in ['valid', 'test']]

train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

### ResNet18 Fine Tuning
We will now use transfer learning on a pretrained net (ResNet18) modifying the last layer to predict the classes of the images.

In [34]:
net = torchvision.models.resnet18(pretrained=True)  #load the net with pretrained weigths
net.fc = nn.Linear(net.fc.in_features, 10)  #modify the fc layer to have an output of 10 classes
nn.init.xavier_normal_(net.fc.weight)  #random initialization of the weights 
nn.init.constant_(net.fc.bias, 0);  # bias

device = 'cuda' if torch.cuda.is_available() else 'cpu'  #we want to move the net on the GPU
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net) # if multiple GPUs use them

Now, we will use:
- Cross Entropy as the $Loss function$
- Adam with Reduced LR as $Optimizer$

In [35]:
lr = 1e-4
wandb.log({'optimizer': 'Adam'})
wandb.log({'lr': lr})
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True, min_lr=1e-5, factor=0.5)

Let us train the net!

In [36]:
# we will train for 10 epochs
epochs = 20
wandb.log({'epochs': epochs})

for epoch in trange(epochs):
    mean_train_losses = []
    mean_valid_losses = []
    valid_acc_list = []
    train_losses = []
    valid_losses = []
    total = 0
    val_acc = 0
    # TRAINING
    for X, y in tqdm(train_iter):
        tr_loss = 0
        X = X.to(device)  #move data on the GPU
        y = y.to(device)
        y_pred = net(X)  #calculate the value using the net, y_pred in this case will be an array of 10 values (probability for each class)
        loss = criterion(y_pred, y)  # calculate the loss wrt the ground truth
        
        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()

        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()

        # Update params
        optimizer.step()
        
        train_losses.append(loss.item())
        
    # VALIDATION
    with torch.no_grad():
        net.eval()  # put network in train mode for Dropout and Batch Normalization
        
        for X, y in tqdm(valid_iter):
            X = X.to(device)
            y = y.to(device)
            y_pred = net(X)
            loss = criterion(y_pred, y)
            _, predicted = torch.max(y_pred, 1)
            val_acc += (y == predicted).sum().float()
            valid_losses.append(loss.item())
            total += len(y)

    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    
    accuracy = 100*val_acc/total
    valid_acc_list.append(accuracy)
    
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
         .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))
    
    wandb.log({'train_loss':mean_train_losses[-1]})
    wandb.log({'val_loss':mean_valid_losses[-1]})
    wandb.log({'val_accuracy':valid_acc_list[-1]})

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 1, train loss : 1.6650, valid loss : 1.3284, valid acc : 65.48%


  0%|          | 0/703 [00:00<?, ?it/s]

...

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 19, train loss : 0.7418, valid loss : 0.9007, valid acc : 82.97%


  0%|          | 0/703 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

epoch : 20, train loss : 0.7337, valid loss : 0.8546, valid acc : 84.27%


### Submission generation
We will generate the submission.csv file based on the test set

In [30]:
preds = []

net.eval()
with torch.no_grad():
    for X, _ in test_iter:
        X = X.to(device)
        preds.extend(net(X).argmax(dim=1).type(torch.int32).cpu().numpy())
ids = list(range(1, len(test_ds)+1))
ids.sort(key=lambda x: str(x))
df = pd.DataFrame({'id': ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.classes[x])
df.to_csv('submission.csv', index=False)